In [6]:
import re
import requests

In [7]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

In [10]:
# 负责采集源码 
def htmls():
    full_list=[]
    for pn in range(0,226,25):     
        base_url = 'https://music.douban.com/top250?start={}'.format(pn)
        print('正在采集{}源码'.format(base_url))
        r = requests.get(base_url,headers=headers)
        html = r.text
        res = makeData(html)
        music_list = sto(res)
        full_list.extend(music_list)
    save_to_dataframe(full_list)
# 信息抽取 
def makeData(html):
    html = repr(html)
    # 1-音乐图片链接地址
    picurl_list = re.findall('<img .*?src="(.*?)"',html)[1:]
    lists = []
    for i in picurl_list:
        sub = 'https://img2'
        res = i.split('.')
        res[0] = sub
        res = '.'.join(res)
        lists.append(res)
    # 2- 专辑名称 
    zj_name = re.findall('<div class="pl2">.*?<a href.*?}\)" >(.*?)</a>',html)
    title_list = []
    for i in zj_name: 
        if '<span' in i:
            main_title = re.findall(r'\\n            (.*?)\\n',i)[0]
            fj_title = re.findall(r'<span style="font-size:12px;">(.*)</span>',i)[0]
            full_info = main_title+'('+fj_title+')'
            title_list.append(full_info)
        else:
            info =  i.split('\\n')[1].strip()
            title_list.append(info)
    
    # # 专辑主要信息 
    singer_info_list=re.findall('<p class="pl">(.*?)</p>',html)
    # #3. 歌手
    singer_list = [i.split('/')[0] for i in singer_info_list]
    # #4. 发型时间 
    times = [i.split('/')[1] for i in singer_info_list]
    # #5.专辑
    zj_type = [i.split('/')[2] for i in singer_info_list]
    # #6.风格
    styles = [i.split('/')[-1] for i in singer_info_list]
    # 7. 评分
    rating = re.findall(r'<span class="rating_nums">(.*?)</span>',html)
    #8. 评价人数
    nums = re.findall(r'<span class="pl">.*?\\n                            (.*?)人评价',html)
    # 9. 专辑详情链接 
    lj_lists = re.findall('<div class="pl2">.*?<a href="(.*?)".*?onclick',repr(html))
    # 返回值
    print(len(lj_lists))
    res = lists,title_list,singer_list,times,zj_type,styles,rating,nums,lj_lists
    return res
# 信息保存
def sto(infoTuple):
    music_list = []  
    picList = infoTuple[0]
    zjList = infoTuple[1]
    singerList = infoTuple[2]
    timeList = infoTuple[3]
    zjstyleList = infoTuple[4]
    stylesList = infoTuple[5]
    ratingList = infoTuple[6]
    numList = infoTuple[7]
    ljList = infoTuple[8]
    for pic,zj,singer,times,zjstyle,styles,rating,nums,lj in zip(picList,
                                                                 zjList,
                                                                 singerList,
                                                                 timeList,
                                                                 zjstyleList,
                                                                 stylesList,
                                                                 ratingList,
                                                                 numList,
                                                                 ljList):
        music_info = {  
                '音乐图片链接地址': pic,  
                '专辑名称': zj,  
                '歌手': singer,  
                '发型时间': times,  
                '专辑': zjstyle,
                '风格': styles,
                '评分': rating,  
                '评价人数': nums,  
                '专辑详情链接': lj
            }
        music_list.append(music_info) 
    return music_list
        # info = pic + ';'+zj+';'+singer+';'+times+';'+zjstyle+';'+styles+';'+rating+';'+nums+';'+lj+'\n'
        # fp.write(info)
        # r = requests.get(pic,headers=headers)
        # content = r.content
        # pic_name = pic.split('/')[-1]
        # fp_pic = open('./db_music_250_pic/'+pic_name,'wb')
        # fp_pic.write(content)
        # fp_pic.close()
        # print('写入并下载图片完成.....SUC!!')
def save_to_dataframe(music_list):  
    df = pd.DataFrame(music_list)  
    df.to_csv('douban_music_top250.csv', index=False, encoding='utf-8-sig')  
    print('数据已保存到douban_music_top250.csv')  
def main():  
    html = htmls()  
    
if __name__ == '__main__':  
    main()

正在采集https://music.douban.com/top250?start=0源码
25
正在采集https://music.douban.com/top250?start=25源码
25
正在采集https://music.douban.com/top250?start=50源码
25
正在采集https://music.douban.com/top250?start=75源码
24
正在采集https://music.douban.com/top250?start=100源码
24
正在采集https://music.douban.com/top250?start=125源码
24
正在采集https://music.douban.com/top250?start=150源码
25
正在采集https://music.douban.com/top250?start=175源码
25
正在采集https://music.douban.com/top250?start=200源码
25
正在采集https://music.douban.com/top250?start=225源码
25
数据已保存到douban_music_top250.csv
